In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join("/home/ubuntu/projects/hindsight/examples/data/ast", '../../..')))
import matplotlib.pyplot as plt
import mplfinance as mpf
import jax
import numpy as np

# Data Management imports
from src import DataManager
from src.data.ast.manager import FormulaManager
from src.data.ast.functions import register_built_in_functions, get_function_context
from src.data.core import prepare_for_jit

import xarray as xr

# Setup core
data_manager = DataManager()

# Load crypto USDT data
market_data = data_manager.load_builtin(
                    "crypto_standard", 
                    "2019-12-30", 
                    "2024-01-01")['crypto_prices'] # Load USDT Spot data from 2019 to 2024 at HOURLY timeframe

# market_data.dt.to_time_indexed().drop_vars(["time_flat", "mask", "mask_indices"]).sel(asset='BTCUSDT').drop_vars(["asset"]).to_dataframe().to_csv('BTCUSDT.csv')
# quit(1)

# Compute some characteristics 
# MA for different periods

# Create a closure that captures the static context
def create_jit_evaluator():
    function_context = get_function_context()
    manager = FormulaManager()
        
    # Capture the static context in the closure
    static_context = {
        "price": "close",
        **function_context
    }
    
    # JIT compile only the dataset processing part
    @jax.jit
    def evaluate_formulas_jit(dataset):
        # Reconstruct the full context inside the JIT function
        context = {
            "_dataset": dataset,
            **static_context
        }
        
        # Multi-configuration evaluation with lag examples
        formula_configs = {
            "sma": [
                {"window": 100},
                {"window": 200}
            ],
        }
        
        return manager.evaluate_bulk(formula_configs, context)
    
    return evaluate_formulas_jit

# Prepare data for JIT, since some vars are not JIT compatible. For example, strings.
market_data_jit, recover = prepare_for_jit(market_data)

evaluate_formulas_jit = create_jit_evaluator()
characteristics = evaluate_formulas_jit(market_data_jit)

/home/ubuntu/projects/hindsight/src/data/core/struct.py:316: AccessorRegistrationWarning: registration of accessor <class 'src.data.core.struct.DataArrayDateTimeAccessor'> under name 'dt' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  @xr.register_dataarray_accessor('dt')


crypto/spot/binance: Attemping to load found cache(/home/suchismit/data/cache/crypto/spot/binance/cdb92fde203902eccc03b671868d4687_2019-01-01_2024-12-31.nc).
crypto/spot/binance: Successfully loaded from /home/suchismit/data/cache/crypto/spot/binance/cdb92fde203902eccc03b671868d4687_2019-01-01_2024-12-31.nc


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
characteristics

<xarray.Dataset> Size: 242MB
Dimensions:       (time_flat: 53568, asset: 265, year: 6, month: 12, day: 31,
                   hour: 24)
Coordinates:
  * time_flat     (time_flat) int64 429kB 0 1 2 3 4 ... 53564 53565 53566 53567
    mask          (time_flat, asset) bool 14MB False False False ... False False
  * month         (month) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
  * asset         (asset) <U13 14kB '1000BONKUSDT' '1000BTTCUSDT' ... 'ZRXUSDT'
    time          (year, month, day, hour) datetime64[ns] 429kB 2019-01-01 .....
  * hour          (hour) int32 96B 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23
    mask_indices  (time_flat, asset) int64 114MB 43670 27388 38824 ... -1 -1 -1
  * year          (year) int32 24B 2019 2020 2021 2022 2023 2024
  * day           (day) int32 124B 1 2 3 4 5 6 7 8 9 ... 24 25 26 27 28 29 30 31
Data variables:
    sma_ww100     (asset, year, month, day, hour) float32 57MB ...
    sma_ww200     (asset, year, month, day, hour) float32 57MB ...

In [6]:
ch_df = characteristics.dt.to_time_indexed().drop_vars(["time_flat", "mask", "mask_indices"]).sel(asset='BTCUSDT').drop_vars(["asset"]).to_dataframe()

In [7]:
mkt_df = market_data.dt.to_time_indexed().drop_vars(["time_flat", "mask", "mask_indices"]).sel(asset='BTCUSDT').drop_vars(["asset"]).to_dataframe()

In [8]:
characteristics.sel(asset='BTCUSDT', year=2020, month=7, day=8, hour=0)['sma_ww100']

<xarray.DataArray 'sma_ww100' ()> Size: 4B
Array(nan, dtype=float32)
Coordinates:
    month    int32 4B 7
    asset    <U13 52B 'BTCUSDT'
    time     datetime64[ns] 8B 2020-07-08
    hour     int32 4B 0
    year     int32 4B 2020
    day      int32 4B 8
Attributes:
    _parent_dataset:  <xarray.Dataset> Size: 413MB\nDimensions:       (year: ...

In [14]:
print(ch_df.loc['2020-07-22T01:00:00'])

sma_ww100   NaN
sma_ww200   NaN
Name: 2020-07-22 00:00:00, dtype: float32


In [15]:
print(mkt_df.loc['2020-07-22T00:00:00'])

open     NaN
high     NaN
low      NaN
close    NaN
volume   NaN
Name: 2020-07-22 00:00:00, dtype: float32


In [11]:
print(market_data.sel(asset='BTCUSDT', year=2020, month=3, day=2, hour=1))

<xarray.Dataset> Size: 911kB
Dimensions:       (time_flat: 53568)
Coordinates:
    year          int32 4B 2020
    month         int32 4B 3
    day           int32 4B 2
    hour          int32 4B 1
    asset         <U13 52B 'BTCUSDT'
    time          datetime64[ns] 8B 2020-03-02T01:00:00
  * time_flat     (time_flat) int64 429kB 0 1 2 3 4 ... 53564 53565 53566 53567
    mask          (time_flat) bool 54kB False False False ... False False False
    mask_indices  (time_flat) int64 429kB 8905 8906 8907 8908 ... -1 -1 -1 -1
Data variables:
    open          float32 4B 8.564e+03
    high          float32 4B 8.576e+03
    low           float32 4B 8.523e+03
    close         float32 4B 8.566e+03
    volume        float32 4B 3.593e+03
